In [ ]:
# =============================================
# 📌 安裝需要的套件（在 Colab / Jupyter 執行）
# =============================================
# 若在本機執行且要使用 GUI 顯示視窗，可改安裝 opencv-python
# 但在伺服器環境通常使用 headless 版本

!pip install keras-facenet mtcnn opencv-python-headless==4.7.0.72 numpy==1.24.6 scikit-learn tqdm opencv-python


In [4]:
# =============================================
# 📌 載入必要套件
# =============================================
import os
import cv2
import pickle
import numpy as np
from tqdm import tqdm
from keras_facenet import FaceNet
from sklearn.metrics.pairwise import euclidean_distances

# 初始化 FaceNet 模型（第一次會自動下載權重）
embedder = FaceNet()

# 使用 OpenCV 內建的 Haar Cascade 進行臉部偵測
haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
detector = cv2.CascadeClassifier(haar_path)

print("🚀 FaceNet 與 Haar Cascade 已成功載入！")



🚀 FaceNet 與 Haar Cascade 已成功載入！


In [3]:
# =============================================
# 📌 擷取臉部 + 調整尺寸
# =============================================
def extract_face_from_image(img, box, required_size=(160, 160)):
    """
    從原始影像中裁切人臉，並調整成 FaceNet 所需輸入大小 160x160。
    box: (x, y, w, h)
    """
    x, y, w, h = box
    face = img[y:y+h, x:x+w]
    if face.size == 0:
        return None
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face = cv2.resize(face, required_size)
    return face

def get_embedding(face_rgb):
    """
    使用 FaceNet 模型將人臉轉成 128 維 embedding。
    """
    emb = embedder.embeddings([face_rgb])
    return emb[0]


In [5]:
# =============================================
# 📌 建立人臉資料庫
# 資料夾結構需為：
# dataset/人名1/xxx.jpg
# dataset/人名2/xxx.jpg
# =============================================

DATASET_DIR = "dataset"
DB_PATH = "face_db.pkl"

def build_database(dataset_dir=DATASET_DIR, db_path=DB_PATH):
    """
    從 dataset 內每個資料夾（人名）建立臉部 embedding 資料庫。
    每個人會取多張照片 embedding 的平均值作為該人的代表向量。
    """
    db = {}

    if not os.path.exists(dataset_dir):
        print(f"⚠️ 找不到資料夾 {dataset_dir}，請先建立 dataset/人名/照片.jpg")
        return db

    persons = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

    for person in persons:
        person_dir = os.path.join(dataset_dir, person)
        files = os.listdir(person_dir)
        embs = []

        print(f"📌 建立 {person} 的臉部資料...")

        for f in tqdm(files):
            path = os.path.join(person_dir, f)
            img = cv2.imread(path)
            if img is None:
                continue
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            boxes = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            if len(boxes) == 0:
                print(f"⚠️ {f} 找不到臉部")
                continue

            face = extract_face_from_image(img, boxes[0])
            emb = get_embedding(face)
            embs.append(emb)

        if len(embs) > 0:
            db[person] = np.mean(embs, axis=0)
            print(f"✔ 已加入 {person}，共 {len(embs)} 張臉部資料！")

    # 儲存資料庫
    with open(db_path, "wb") as f:
        pickle.dump(db, f)

    print("💾 資料庫建立完成並已儲存到", db_path)
    return db

# 執行建立資料庫（請先把圖片放進 dataset 資料夾）
db = build_database()


📌 建立 whale 的臉部資料...


  0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


100%|██████████| 1/1 [00:04<00:00,  4.94s/it]

✔ 已加入 whale，共 1 張臉部資料！
💾 資料庫建立完成並已儲存到 face_db.pkl


In [6]:
# =============================================
# 📌 載入人臉資料庫（若已建立過）
# =============================================
def load_database(db_path=DB_PATH):
    if os.path.exists(db_path):
        with open(db_path, "rb") as f:
            return pickle.load(f)
    return {}

db = load_database()
print("📚 目前資料庫人員：", list(db.keys()))


📚 目前資料庫人員： ['whale']


In [7]:
# =============================================
# 📌 辨識單一人臉
# threshold 為可調參數，越小代表要求越嚴格（人臉越像才算同一人）
# =============================================
def recognize_face(face_rgb, db, threshold=0.9):
    if len(db) == 0:
        return "Unknown", None

    emb = get_embedding(face_rgb).reshape(1, -1)
    names = list(db.keys())
    mats = np.vstack([db[n] for n in names])

    dists = euclidean_distances(emb, mats)[0]
    idx = np.argmin(dists)
    dist = dists[idx]

    if dist <= threshold:
        return names[idx], float(dist)
    else:
        return "Unknown", float(dist)


In [8]:
# =============================================
# 📌 即時攝影機人臉辨識
# use_headless = True 時不會開視窗（適合 Colab / Server）
# use_headless = False 可開視窗顯示畫面（需本機環境）
# =============================================

use_headless = True   # 若你在本機想看到視窗 -> 改成 False

def run_webcam(db, threshold=0.9, cam_id=0, max_frames=500):
    cap = cv2.VideoCapture(cam_id)
    if not cap.isOpened():
        print("❌ 找不到攝影機。請確認 camera ID 是否正確。")
        return

    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        boxes = detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in boxes:
            face = extract_face_from_image(frame, (x, y, w, h))
            if face is None:
                continue

            name, dist = recognize_face(face, db, threshold)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 2)
            cv2.putText(frame, f"{name}({dist:.2f})", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

        if not use_headless:
            cv2.imshow("Real-time Face Recognition", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # 無 GUI 時，每 30 幀存一張 snapshot
            if frame_count % 30 == 0:
                cv2.imwrite(f"snapshot_{frame_count}.jpg", frame)
                print(f"📸 已存擋 snapshot_{frame_count}.jpg")

        frame_count += 1

    cap.release()
    if not use_headless:
        cv2.destroyAllWindows()

print("🎉 即時辨識程式已準備完成！執行 → run_webcam(db)")


🎉 即時辨識程式已準備完成！執行 → run_webcam(db)


In [10]:
run_webcam(db)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
📸 已存擋 snapshot_0.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━

KeyboardInterrupt: 

--vs2--

In [21]:
# 建立模型
import os
import cv2
import pickle
import numpy as np
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer

# ========================================
# 初始化 FaceNet 模型
# ========================================
embedder = FaceNet()

# L2 正規化，用來將 embedding 正規化
l2_normalizer = Normalizer('l2')

# ========================================
# 設定資料集路徑
# ========================================
DATASET_DIR = "dataset"   # 修改成你自己的資料夾名稱

# 用來存放 { "name": [embedding vectors] }
face_db = {}

# Haar Cascade 用於臉部偵測
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# ========================================
# 讀取每個人的照片並建立 embeddings
# ========================================
for person in os.listdir(DATASET_DIR):
    person_dir = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"正在處理：{person}")
    embeddings = []

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f"無法讀取：{img_path}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            print(f"沒有偵測到臉：{img_path}")
            continue

        x, y, w, h = faces[0]
        face = img[y:y+h, x:x+w]

        # FaceNet 模型輸入格式
        face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        embedding = embedder.embeddings([face_rgb])[0]

        embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]
        embeddings.append(embedding)

    if len(embeddings) > 0:
        face_db[person] = embeddings

# ========================================
# 儲存 embedding 資料庫
# ========================================
with open("face_db.pkl", "wb") as f:
    pickle.dump(face_db, f)

print("🔍 人臉資料庫建立完成，已儲存為 face_db.pkl")


正在處理：s1254001


Exception ignored in: <function AtomicFunction.__del__ at 0x0000029D602AAC20>
Traceback (most recent call last):
  File "c:\Users\jxu27\anaconda3\envs\project3_py310\lib\site-packages\tensorflow\python\eager\polymorphic_function\atomic_function.py", line 286, in __del__
    def __del__(self):
KeyboardInterrupt: 


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
正在處理：s1254006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
正在處理：s1254019
沒有偵測到臉：dataset\s1254019\img1.jpg
正在處理：s1254029
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
正在處理：s1254040
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
正在處理：s1254042
沒有偵測到臉：dataset\s1254042\img1.jpg
正在處理：s1254051
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
正在處理：s1261115
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
正在處理：s1267011
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
🔍 人臉資料庫建立完成，已儲存為 face_db.pkl


In [19]:
import cv2
import pickle
import numpy as np
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer

# =============================
# 載入 embedding 資料庫
# =============================
with open("face_db.pkl", "rb") as f:
    face_db = pickle.load(f)

# FaceNet 模型
embedder = FaceNet()
l2_normalizer = Normalizer('l2')

# Haar Cascade 臉部偵測
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# =============================
# 計算 embedding 相似度（L2 距離）
# =============================
def recognize_face(face_img):
    """輸入臉部圖片，返回辨識到的名稱（以最小距離為準）"""

    # 轉成 FaceNet 格式
    face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    embedding = embedder.embeddings([face_rgb])[0]
    embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]

    best_match = "Unknown"
    best_distance = 1e6
    threshold = 0.95  # 距離小於此值視為同一人

    for person, embeds in face_db.items():
        for known_emb in embeds:
            dist = np.linalg.norm(embedding - known_emb)
            if dist < best_distance:
                best_distance = dist
                best_match = person

    if best_distance < threshold:
        return best_match
    else:
        return "Unknown"

# =============================
# 啟動 Webcam 即時偵測
# =============================
cap = cv2.VideoCapture(0)

print("🎥 正在啟動攝影機... 按 Q 可退出")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        name = recognize_face(face_img)

        # 繪製辨識結果
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, name, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Real-Time Face Recognition", frame)

    # 按 Q 離開
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 正在啟動攝影機... 按 Q 可退出
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━

--v3 final--

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer
import cv2
import numpy as np
import os
import pickle


# ========================================
# 啟動 FaceNet
# ========================================
embedder = FaceNet()
l2_normalizer = Normalizer("l2")

# ========================================
# 臉部偵測器（Haar）
# ========================================
face_detector = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# ========================================
# 建立資料增強器
# ========================================
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# ========================================
# 資料集路徑
# ========================================
DATASET_DIR = "dataset"
face_db = {}


# ========================================
# 顯示資料擴充圖像 + 回傳 FaceNet embeddings
# ========================================
def augment_and_embed(face_img, name):
    """輸入臉部影像 → 顯示增強後的圖片 → 產生 embeddings"""

    # FaceNet 輸入大小 (160x160)
    face = cv2.resize(face_img, (160, 160))
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)

    x = face.reshape((1,) + face.shape)

    embeddings = []

    print(f"🔄 正在為 {name} 進行資料擴充...")

    # 產生 5 張增強影像
    i = 0
    for batch in datagen.flow(x, batch_size=1):
        augmented = batch[0].astype("uint8")

        # ===============================
        # 顯示擴充後的圖片
        # ===============================
        show_img = cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
        cv2.imshow(f"Augmented Images - {name}", show_img)
        cv2.waitKey(300)   # 每張顯示 300ms

        # 產生 embedding
        embedding = embedder.embeddings([augmented])[0]
        embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]
        embeddings.append(embedding)

        i += 1
        if i >= 5:
            break

    cv2.destroyAllWindows()
    return embeddings


# ========================================
# 建立資料庫（含資料增強 + 顯示）
# ========================================
for person in os.listdir(DATASET_DIR):
    person_dir = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_dir):
        continue

    print(f"📌 正在處理：{person}")
    all_embeddings = []

    for img_name in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_detector.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            continue

        x, y, w, h = faces[0]
        face_img = img[y:y+h, x:x+w]

        # 原始 embedding
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        original_embedding = embedder.embeddings([face_rgb])[0]
        original_embedding = l2_normalizer.transform(original_embedding.reshape(1, -1))[0]
        all_embeddings.append(original_embedding)

        # 加入資料增強後的 embeddings（同時顯示）
        augmented_embeddings = augment_and_embed(face_img, person)
        all_embeddings.extend(augmented_embeddings)

    face_db[person] = all_embeddings


# ========================================
# 儲存 face_db.pkl
# ========================================
with open("face_db.pkl", "wb") as f:
    pickle.dump(face_db, f)

print("✅ 已完成資料擴充 + 顯示 + 建立 face_db.pkl！")



📌 正在處理：s1254001
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
🔄 正在為 s1254001 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
🔄 正在為 s1254001 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
📌 正在處理：s1254006
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
🔄 正在為 s1254006 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
📌 正在處理：s1254016
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
🔄 正在為 s1254016 進行資料擴充...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/st

In [2]:
import cv2
import pickle
import numpy as np
import threading
from tkinter import *
from PIL import Image, ImageTk
from keras_facenet import FaceNet
from sklearn.preprocessing import Normalizer


# =============================
# 載入人臉 embedding 資料庫
# =============================
with open("face_db.pkl", "rb") as f:
    face_db = pickle.load(f)

# FaceNet 模型
embedder = FaceNet()
l2_normalizer = Normalizer("l2")

# Haar Cascade 臉部偵測
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")


# =============================
# 人臉辨識函式
# =============================
def recognize_face(face_img):
    """輸入裁切後的臉，回傳最佳配對名稱"""
    face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    embedding = embedder.embeddings([face_rgb])[0]
    embedding = l2_normalizer.transform(embedding.reshape(1, -1))[0]

    best_match = "Unknown"
    best_distance = 999
    threshold = 0.95  

    for person, embeds in face_db.items():
        for known_emb in embeds:
            dist = np.linalg.norm(embedding - known_emb)
            if dist < best_distance:
                best_distance = dist
                best_match = person
    
    return best_match if best_distance < threshold else "Unknown"


# =============================
# 啟動攝影機 + Tkinter 視窗顯示
# =============================
cap = cv2.VideoCapture(0)

# 建立 Tkinter 主視窗
root = Tk()
root.title("即時人臉辨識系統")

# Tkinter Label 用於顯示影像
video_label = Label(root)
video_label.pack()


def update_frame():
    """讀取攝影機畫面，辨識後顯示於 Tkinter 視窗"""
    ret, frame = cap.read()
    if not ret:
        root.after(10, update_frame)
        return

    # 灰階用於 Haar 偵測
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)

    # 對每個偵測到的臉進行辨識
    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        name = recognize_face(face_img)

        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, name, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # 轉換成 Tkinter 可顯示的格式
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)

    # 更新 Tkinter Label 內容
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    # 每 10 ms 更新一次
    root.after(10, update_frame)


def on_closing():
    """關閉視窗時釋放攝影機與資源"""
    cap.release()
    root.destroy()


# 綁定關閉事件
root.protocol("WM_DELETE_WINDOW", on_closing)

# 開始更新畫面
update_frame()

# 啟動 Tkinter 主迴圈
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115m